In [19]:
import pandas as pd
from IPython.display import Markdown
import plotly.graph_objects as go

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.model import build_model
from tb_incubator.plotting import set_plot_label, display_plot, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, get_param_table, load_genexpert_util, load_report_section

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")

## Introduction

In [20]:
#load_report_section('reporting.yaml', 'introduction')
#load_report_section('reporting.yaml', 'introduction2')

In [21]:
targets = load_targets()
indicator_names = const.indicator_names
compartments = const.compartments

## Model construction

In [22]:
param_info = load_param_info()
params = param_info["value"]

In [23]:
model, desc = build_model(params, xpert_sensitivity=False, covid_effects=True)

#Markdown(desc)

In [24]:
calib_params = {
    'contact_rate': 36.203285817215914,
    'progression_multiplier': 0.7975952842355304,
    'detection_reduction': 0.5707673479232517,
    'post_covid_improvement': 1.9226694724100577,
    'sustained_improvement': 5.1324558252247225,
}

In [25]:
model.run(params | calib_params)

## Results

In [26]:
outs = model.get_derived_outputs_df()

In [ ]:
load_report_section('reporting.yaml', 'result-compartment')

In [28]:
file_prefix = "id_noxpert"

In [ ]:
fig = outs[[f"comp_size_{c}" for c in compartments]].plot.area()
#fig.add_scatter(x=targets["census_pop"].index, y=targets["census_pop"], mode="markers", name="Census data")
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
set_plot_label(fig, indicator_names, "Compartment size")
fig

#fig.update_xaxes(range=[2000, 2035])
display_plot(fig, f"{file_prefix}_comp_size", "svg")


In [ ]:
load_report_section('reporting.yaml', 'result-notifprev')

In [ ]:
#| label: fig-notifprev
#| fig-cap: "Comparison between model outputs and local data for calibration targets. The red dots represent the actual data from all periods, while the green dots indicate the data specifically used for calibration targets. For prevalence, all actual data is used as the calibration target."
#| fig-subcap: 
#|   - "TB notifications"
#|   - "TB prevalence"
#| layout-ncol: 2
#| fig-pos: '!ht'

fig = plot_model_vs_actual(outs, 
                     targets["notif"],
                     "notification",
                     "Notification",
                     "",
                     "Actual data (All periods)", actual_color="gray")
fig.update_layout(showlegend=True)

# Add target points as a scatter trace
fig.add_trace(
    go.Scatter(
        x=targets["notif2000"].index,  # Filter for 2013-2023
        y=targets["notif2000"],
        mode='markers',  # Only show markers, no lines
        marker=dict(
            color='hotpink',  
        ),
        name='Target'  # This will show in legend if you enable it
    )
)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[2000, 2024])
display_plot(fig, f"{file_prefix}_notif", "svg")

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["prevalence"],
                     "prevalence",
                     "Prevalence",
                     "",
                     "Actual data", actual_color="gray")
fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[2000, 2024])
display_plot(fig, f"{file_prefix}_prevalence", "svg")

In [ ]:
fig = plot_model_vs_actual(outs, 
                     targets["incidence"],
                     "incidence",
                     "Incidence (per 100,000)",
                     "",
                     "Actual data", actual_color="gray")
fig.update_layout(showlegend=True)

fig.add_trace(
    go.Scatter(
        x=[2030.],  
        y=[65.0],
        mode='markers',  
        marker=dict(
            color='hotpink',  
        ),
        name='2030 Target'  # This will show in legend if you enable it
    )
)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.update_xaxes(range=[2000, 2035])
display_plot(fig, f"{file_prefix}_incidence", "svg")

In [ ]:
fig = outs["percentage_latent"].plot()
fig.update_layout(showlegend=False)
fig.update_xaxes(range=[2000, 2035], title_text="Year")

set_plot_label(fig, indicator_names, "Percentage of latent TB infection (%)")
fig
display_plot(fig, f"{file_prefix}_ltbi", "svg")

In [35]:
#load_report_section('reporting.yaml', 'result-conclusion')

\newpage
## Supplementary

In [36]:
#| label: tbl-params
#| tbl-cap: Model parameters
#| tbl-cap-location: top

fixed_param_table = get_param_table(param_info)
#Markdown(fixed_param_table.to_markdown())


\newpage
## References